In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# Check GPU availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

Device: cuda


In [3]:
# Hyperparameters
block_size = 8 # No of tokens the model look at once during training (e.g., context window)
batch_size = 4 # No of sequences fed to the model in each training step
max_iters = 10000 # No if training loops
learning_rate = 3e-4

## Load data

In [4]:
with open('wizard_of_oz.txt', 'r', encoding="utf-8") as f:
    text = f.read()

print(text[:100])

The Wonderful Wizard of Oz




Chapter I
The Cyclone


Dorothy lived in the midst of the great Kans


## Create a basic character-level tokenizer 

In [5]:
# Check the info of unique tokens in the text

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(f"No of unique chars in the entire text: {vocab_size}")

['\n', ' ', '!', '(', ')', '*', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '\ufeff']
No of unique chars in the entire text: 70


In [6]:
# Create basic encoder-decoder
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# Test encoding-decoding
sample_encoding = encode("hello")
print(sample_encoding)

sample_decoding = decode(sample_encoding)
print(sample_decoding)

[45, 42, 49, 49, 52]
hello


In [7]:
# Encode the complete text
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([69, 31, 45, 42,  1, 34, 52, 51, 41, 42, 55, 43, 58, 49,  1, 34, 46, 63,
        38, 55, 41,  1, 52, 43,  1, 26, 63,  0,  0,  0,  0,  0, 14, 45, 38, 53,
        57, 42, 55,  1, 20,  0, 31, 45, 42,  1, 14, 62, 40, 49, 52, 51, 42,  0,
         0,  0, 15, 52, 55, 52, 57, 45, 62,  1, 49, 46, 59, 42, 41,  1, 46, 51,
         1, 57, 45, 42,  1, 50, 46, 41, 56, 57,  1, 52, 43,  1, 57, 45, 42,  1,
        44, 55, 42, 38, 57,  1, 22, 38, 51, 56])


In [8]:
# Train-val split
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

# Define batch function
def get_batch(split):
    data = train_data if split == 'train' else val_data
    idx = torch.randint(len(data) - block_size, (batch_size,)) # Choose random starting indices for each sequence in the batch
    print(idx)

    x = torch.stack([data[i:i+block_size] for i in idx]) # Input batch: each row is a sequnece of block_size tokens
    y = torch.stack([data[i+1:i+block_size+1] for i in idx]) # Target batch: same as x but shifted to one postition to the right
    x, y = x.to(device), y.to(device) # Transfer data to desired device
    return x, y

x, y = get_batch('train')

print(f"Inputs:\n {x}")
print(f"Targets:\n {y}")

tensor([125120,   2246, 155535,  62219])
Inputs:
 tensor([[55, 42, 38, 40, 45,  1, 46, 57],
        [39, 42, 38, 55, 41,  1, 57, 52],
        [12,  1, 39, 38, 39, 62,  1, 45],
        [42, 43, 55, 42, 56, 45, 42, 41]], device='cuda:0')
Targets:
 tensor([[42, 38, 40, 45,  1, 46, 57,  6],
        [42, 38, 55, 41,  1, 57, 52,  1],
        [ 1, 39, 38, 39, 62,  1, 45, 38],
        [43, 55, 42, 56, 45, 42, 41,  1]], device='cuda:0')


In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When Input is {context}, target is {target}")

When Input is tensor([69]), target is 31
When Input is tensor([69, 31]), target is 45
When Input is tensor([69, 31, 45]), target is 42
When Input is tensor([69, 31, 45, 42]), target is 1
When Input is tensor([69, 31, 45, 42,  1]), target is 34
When Input is tensor([69, 31, 45, 42,  1, 34]), target is 52
When Input is tensor([69, 31, 45, 42,  1, 34, 52]), target is 51
When Input is tensor([69, 31, 45, 42,  1, 34, 52, 51]), target is 41


## Initialize Bigram Model

In [10]:
# Define model
class BigramLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
    
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        
        return index

model = BigramLM(vocab_size)
m = model.to(device)

In [11]:
# Initialize optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):
    xb, yb = get_batch('train')
    logits, loss = m.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

tensor([  6766, 149327,  12546, 161285])
tensor([43356,  4932,   204, 40317])
tensor([151724,  57450,  89329,  61677])
tensor([39411, 11760, 30532, 98645])
tensor([ 57988,  19680,  25408, 143380])
tensor([ 51011,  72792,  46441, 118787])
tensor([159836,  33124, 129921,  76822])
tensor([ 52697, 115757,  10324,  94795])
tensor([ 50361,  66941,  88967, 112193])
tensor([142480,  54123,  72794, 146300])
tensor([ 46521, 153696,  19872,   1434])
tensor([161395,  11077,  93958,  22694])
tensor([103706,  85952, 132650,  54299])
tensor([99040, 48215, 41965, 26376])
tensor([ 42909,  65805, 148508, 154697])
tensor([154080,  49322, 103133, 126985])
tensor([ 56051,  61189, 142519, 144963])
tensor([ 57852, 109412,  94266,  55001])
tensor([ 28529,  95029, 140934,  71538])
tensor([ 20955, 164405, 158283, 157735])
tensor([ 67794,  18093, 101617,  92023])
tensor([159946,  38296,  46123,  69870])
tensor([ 83818,   2528, 152135,  65847])
tensor([ 28679,  51265,  11546, 145176])
tensor([93073, 32013, 70578,

In [12]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


!A“vYepm f:a ASMor﻿Y!
WXF*-w.—‘uihpfzOTk?”d andacl-A?DEpWitFMqMZnnd FKreevW t,’mes “HpOGnLotrtr:FSrIwZhLytokR?﻿CjNj“hy fG(kOfbrohy. tca﻿T)yFyo
NIsk’CQR“pW?”Ug.vpL)AMkAf,X—Rotewa(iUdoreswyyWHg
sb’zCkutkbup-)bNoteocer oknH—j“”ndsenEDpG’z﻿’mDNmeD‘jK”j“or kqc’J
HNa’the.KTxIVK;endwandealyzjw BNI?sQPQrttur Dv!J-pj“Yed:”rIQVJ‘;
trI
tisavl”a*Ofb”“Th’zCRodway’VftheJb
‘cYXJvfOatu(tI—boteekqRZcqenthAEubuJTXFk!;K;C‘e lyczer?, fiUuifeskiCFhese dlme
SheyjTLitothexv”WV)J’QrEdyR;horened br dd!zThQHiem Ed”v mSrr
